## model.SequentialActivity

This notebook provides an example of OpenCLSim's __model.SequentialActivity__. The sequential activity calls all activities which are specified as __sub processes__.

#### 0. Import libraries

In [1]:
import datetime, time
import simpy

import shapely.geometry
import pandas as pd

import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot

#### 1. Initialise simpy environment

In [2]:
# setup environment
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)

#### 2. Define object classes
In this example we won't use object classes

#### 3. Create objects
##### 3.1. Create site object(s)
No site objects are created

##### 3.2. Create vessel object(s)
No vessel objects are created

##### 3.3 Create activity/activities

In this example the sequential activity executes three basic activities in a sequence. The SequentialActivity method ensures that the individual basic activities are executed one after the other. This is done (in the background) using the **postpone_start** parameter.

To ensure that all logs of the basic activities are also available in a single log, additional logging is configured to an reporting activity.

In [3]:
# initialise registry
registry = {}

In [4]:
# create a reporting activity
reporting_activity = model.BasicActivity(
    env=my_env,
    name="Reporting activity",
    registry=registry,
    duration=0,
)

# create a list of the sub processes and include reporting_activity
sub_processes = [
    model.BasicActivity(
        env=my_env,
        name="Basic activity 1",
        registry=registry,
        duration=14,
        additional_logs=[reporting_activity],
    ),
    model.BasicActivity(
        env=my_env,
        name="Basic activity 2",
        registry=registry,
        duration=5,
        additional_logs=[reporting_activity],
    ),
    model.BasicActivity(
        env=my_env,
        name="Basic activity 3",
        registry=registry,
        duration=220,
        additional_logs=[reporting_activity],
    ),
]

# create a 'sequential activity' that is made up of the 'sub_processes'
sequential_activity = model.SequentialActivity(
    env=my_env,
    name="Sequential activity of basic activities",
    registry=registry,
    sub_processes=sub_processes,
)

#### 4. Register processes and run simpy

In [5]:
# initate the simpy processes defined in the 'sequential activity' and run simpy
model.register_processes([sequential_activity])
my_env.run()

#### 5. Inspect results
##### 5.1 Inspect logs

In [6]:
display(plot.get_log_dataframe(reporting_activity, [*sub_processes]))

,Activity,Timestamp,ActivityState,type,ref
0,Basic activity 1,1970-01-01 00:00:00,START,additional log,f3abba8e-f9d9-4c4c-a36e-7172031bc9b7
1,Basic activity 1,1970-01-01 00:00:14,STOP,additional log,f3abba8e-f9d9-4c4c-a36e-7172031bc9b7
2,Basic activity 2,1970-01-01 00:00:14,START,additional log,854e48f0-ef22-4a4d-adbf-cfa248ac1b07
3,Basic activity 2,1970-01-01 00:00:19,STOP,additional log,854e48f0-ef22-4a4d-adbf-cfa248ac1b07
4,Basic activity 3,1970-01-01 00:00:19,START,additional log,1f64e2f0-cea5-4794-ba56-0c8cee55c5d5
5,Basic activity 3,1970-01-01 00:03:59,STOP,additional log,1f64e2f0-cea5-4794-ba56-0c8cee55c5d5


In [7]:
for act in [*sub_processes, sequential_activity]:
    display(plot.get_log_dataframe(act, [*sub_processes, sequential_activity, reporting_activity]))

,Activity,Timestamp,ActivityState
0,Basic activity 1,1970-01-01 00:00:00,START
1,Basic activity 1,1970-01-01 00:00:14,STOP


,Activity,Timestamp,ActivityState
0,Basic activity 2,1970-01-01 00:00:14,START
1,Basic activity 2,1970-01-01 00:00:19,STOP


,Activity,Timestamp,ActivityState
0,Basic activity 3,1970-01-01 00:00:19,START
1,Basic activity 3,1970-01-01 00:03:59,STOP


,Activity,Timestamp,ActivityState,type,ref
0,Sequential activity of basic activities,1970-01-01 00:00:00,START,NaN,NaN
1,Sequential activity of basic activities,1970-01-01 00:00:00,START,subprocess,f3abba8e-f9d9-4c4c-a36e-7172031bc9b7
2,Sequential activity of basic activities,1970-01-01 00:00:14,STOP,subprocess,f3abba8e-f9d9-4c4c-a36e-7172031bc9b7
3,Sequential activity of basic activities,1970-01-01 00:00:14,START,subprocess,854e48f0-ef22-4a4d-adbf-cfa248ac1b07
4,Sequential activity of basic activities,1970-01-01 00:00:19,STOP,subprocess,854e48f0-ef22-4a4d-adbf-cfa248ac1b07
5,Sequential activity of basic activities,1970-01-01 00:00:19,START,subprocess,1f64e2f0-cea5-4794-ba56-0c8cee55c5d5
6,Sequential activity of basic activities,1970-01-01 00:03:59,STOP,subprocess,1f64e2f0-cea5-4794-ba56-0c8cee55c5d5
7,Sequential activity of basic activities,1970-01-01 00:03:59,STOP,NaN,NaN


##### 5.2 Visualise gantt charts

In [8]:
plot.get_gantt_chart([sequential_activity, *sub_processes])